In [ ]:
# Imports 
import pandas as pd
import itertools 
import networkx as nx
# import matplotlib.pyplot as plt
# import pytrends
# import musicbrainzngs as m
import numpy as np
import requests
import sys

# from pytrends.request import TrendReq
from os import path
from PIL import Image
# from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
def get_access_token(client_id, client_secret):
    """
    Get the access token from Spotify
    """
    body_params = {'grant_type': "client_credentials"}
    url = 'https://accounts.spotify.com/api/token'

    response = requests.post(url, data=body_params, auth=(client_id, client_secret))
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        sys.exit("Failed to get access token. Is your client_id and client_secret correct?")

In [ ]:
access_ = get_access_token("","")

In [ ]:
access = "Bearer " + access_

In [ ]:
subgenres_popularity = dict({e:0 for e in [i for i in list(aggregate_genres.values()) for i in i]})

In [ ]:
for index, row in df.loc[pd.notna(df['subgenres'])].iterrows():
    for subg in row['subgenres']:
        try : 
            subgenres_popularity[subg] += 1/row['rank']
        except KeyError:
            continue
        
    
    # print(index, row['rank'], row['subgenres'])

In [ ]:
# subgenres_popularity = {key:value for (key,value) in subgenres_popularity.items() if value>0}

In [ ]:
separate_artists(artist_genres_df)

In [ ]:
def find_artist_genres(artist_name):
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': access}
    params = {
        'q': artist_name,
        'type': 'artist',
        'limit': '1',
    }

    response = requests.get('https://api.spotify.com/v1/search', params=params, headers=headers)
    
    return response.json()['artists']['items'][0]['genres']

In [ ]:
find_artist_genres("Lady Gaga")

In [ ]:
import ciso8601
import time
import re
def separate_artists(df):
    split_string = ' and |, | AND | & | featuring | FEATURING | x | X | feat | ft |,| with | feauring | feat. | / '
    ret_list = []
    res_check = set()
    for index, colab in df.iterrows():
        date = ciso8601.parse_datetime(colab['date'])
        res = [time.mktime(date.timetuple())]
        artists = re.split(split_string, colab['artist'], flags=re.IGNORECASE)
        if tuple(artists) not in res_check:
            res_check.add(tuple(artists))
            res.append(artists)
            ret_list.append(res)
    return ret_list

In [ ]:
df['song']

In [ ]:
df = pd.read_csv("../data/billboard.csv", index_col = 0)
df = df[df.date >= "2000-01-01"]

separated_artists = separate_artists(df)
collabs = [y for [x,y] in separated_artists if len(y)>1]

In [ ]:
df = pd.read_csv("../docs/src/chartview/data/data.csv")

In [ ]:
def find_uri(artist, track):
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': access,}

    params = {
         # 'q': '"' + track + '" "' + artist + '"',
         'q': "track: " + track + " artist:" + artist,
        'type': 'track',
        'limit': '1',
        'market': 'US'
    }

    response = requests.get('https://api.spotify.com/v1/search', params=params, headers=headers)
    return response.json()['tracks']['items'][0]['uri'].split(":")[-1]

In [ ]:
df

In [ ]:
for i, row in df.iterrows():
    if row['_uri'] == "Unknown":
        try: 
            uri = find_uri(row["_song"], row["_artist"] )
            df.at[i, '_uri'] = uri
        except IndexError:
            print(row["_song"], row["_artist"])

In [ ]:
df.to_csv("../docs/src/chartview/data/data.csv", index = False)

In [ ]:
"spotify:track:5dXWFMwD7I7zXsInONVl0H".split(":")[-1]

In [ ]:
for i in collabs:
    try :
        i.remove('R. Kelly')
    except ValueError:
        pass

In [ ]:
unique_artists = list(np.unique([artist for l in collabs for artist in l]))

In [ ]:
unique_artists;
artists_by_id = dict({a:b for a,b in zip(unique_artists,list(range(0,len(unique_artists))))})
artists_by_id = {key:[value,0.0] for (key,value) in artists_by_id.items()}
artists_by_id.pop("R. Kelly")

In [ ]:
for l in collabs:
    for artist in l:
        artists_by_id[artist][1] +=1

In [ ]:
top100=sorted(artists_by_id.items(), key=lambda x: x[1][1], reverse=True)[:100];
top100=dict(top100);
top100

In [ ]:
find_artist_genres('Shakira')

In [ ]:
def determine_genre(subgenres) :
    count_genres = dict(zip(aggregate_genres.keys(), np.zeros(len(aggregate_genres.keys()))))
    find_one = False
    for subgenre in subgenres :
        for genre, subgrs in aggregate_genres.items():
            if subgenre in subgrs:
                find_one = True
                count_genres[genre] +=1
    if (find_one):
        return sorted(count_genres.items(), key=lambda x: x[1])[-1][0]
    else :
        return np.NaN

In [ ]:
# Source : https://github.com/kevinschaich/billboard but completed with genres suggested by Spotify API
aggregate_genres = {
    "rock": ["symphonic rock", "jazz-rock", "heartland rock", "rap rock", "garage rock", 
           "folk-rock", "roots rock", "adult alternative pop rock", "rock roll", "punk rock", 
           "arena rock", "pop-rock", "glam rock", "southern rock", "indie rock", "funk rock", 
           "country rock", "piano rock", "art rock", "rockabilly", "acoustic rock", "progressive rock", 
           "folk rock", "psychedelic rock", "rock & roll", "blues rock", "alternative rock", 
           "rock and roll", "soft rock", "rock and indie", "hard rock", "pop/rock", "pop rock", 
           "rock", "classic pop and rock", "psychedelic", "british psychedelia", "punk", "metal", "heavy metal", 
             "brutal death metal", 'deep talent show'],
"alternative/indie": ["adult alternative pop rock", "alternative rock", "alternative metal", 
                       "alternative", "lo-fi indie", "indie", "indie folk", "indietronica", 
                       "indie pop", "indie rock", "rock and indie"],
"electronic/dance": ['big room', 'edm', "dance and electronica", "pop dance", "electro house", "electronic", "electropop", 
                      "progressive house", "hip house", "house", "eurodance", "dancehall", "dance", "funk carioca"],
"soul": ["psychedelic soul", "deep soul", "neo-soul", "neo soul", "southern soul", 
          "smooth soul", "blue-eyed soul", "soul and reggae", "soul", "funk"],
"classical/soundtrack": ["classical", "orchestral", "film soundtrack", "composer"],
"pop": ["country-pop","urban contemporary", "latin pop", "classical pop", "pop-metal", "orchestral pop", 
         "instrumental pop", "indie pop", "sophisti-pop", "pop punk", "pop reggae", "noah cyrus"
         "britpop", "traditional pop", "power pop", "sunshine pop", "baroque pop", "alternative r&b",
         "synthpop", "art pop", "teen pop", "psychedelic pop", "folk pop", "country pop", "pop soul",
         "pop and chart", "dance-pop", "pop", "top 40", "american idol", 'breakeven', 'm2m', 'dance pop'],
"hip-hop/rap": ["conscious hip hop", "east coast hip hop", "hardcore hip hop", 'oakland hip hop', 'pop rap'
                 "west coast hip hop", "hiphop", "southern hip hop", "hip-hop", 
                 "hip hop", "hip hop rnb and dance hall", "contemporary r b", 
                 "gangsta rap", 'gangster rap', "rapper", "rap", "mumble rapper", 'chicago rap', 'melodic rap', 'brooklyn drill', 'drill',
               'baton rouge rap', 'trap', 'southern hip hop', 'miami hip hop', 'dirty south rap'],
"rnb": ["rnb", "rhythm & blues","r&b", "blues", "contemporary rnb", 
                 "contemporary r&b", "rhythm and blues"],
"disco": ["disco"],
"swing":  ["swing"],
"folk": ["contemporary folk", "folk", "bachata", "romanian"],
"country": ["country rock", "country-pop", "country pop", "contemporary country", "country"],
"jazz": ["vocal jazz", "jazz", "jazz-rock"],
"religious": ["christian", "christmas music", "gospel"],
"blues": ["delta blues", "rock blues", "urban blues", "electric blues",
           "acoustic blues", "soul blues", "country blues", "jump blues", 
           "classic rock", "blues rock", "jazz and blues", "piano blues", 
           "british blues", "british rhythm & blues", "rhythm and blues", "blues", "blues rock", "rhythm & blues"],
"reggae": ["reggae", "reggae fusion", "roots reggae", "reggaeton", "pop reggae", "soul and reggae"]}

In [ ]:
for key, elem in top100.items():
    elem.append(determine_genre(find_artist_genres(key)))

In [ ]:
find_artist_genres("David Guetta")

In [ ]:
top100

In [ ]:
selected_genres = []

for k,v in top100.items():
    if ((v[2] not in selected_genres)):
        selected_genres.append(v[2])

In [ ]:
selected_genres

In [ ]:
top100['David Guetta'] = [410, 21.0, 'electronic/dance']

In [ ]:
top100

In [ ]:
find_artist_genres("Usher")

In [ ]:
selected_genres = []

for k,v in top100.items():
    if ((v[2] not in selected_genres)):
        selected_genres.append(v[2])
        
selected_genres

In [ ]:
import os 

f = open("nodes.json", "w")

f.write("[")

color = ""
for artist, elems in top100.items():
    genre = elems[2]

    if genre == 'hip-hop/rap':
         color = "#001219"
    elif genre == 'pop':
        color = "#005f73"
    elif genre == 'rnb':
        color = "#e9d8a6"        
    elif genre == 'reggae':
        color = "#ee9b00"
    elif genre == 'electronic/dance':
        color = "#bb3e03"
    elif genre == 'rock':
        color = "#9b2226"
    
    f.write('{"id": ' + str(elems[0]) )
    f.write(', "label": "' + artist  )
    f.write('", "group": "' + elems[2] + '", "color": "' + color +'"},\n')

f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')
f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')
f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')

f.write("]")
f.close()

In [ ]:
f = open("edges.json", "w")

f.write("[")

for pair in list(itertools.combinations(list(top100.keys()), 2)):
    for collab in collabs:
        if pair[0] in collab and pair[1] in collab:
                f.write('{"from": ' + str(artists_by_id[pair[0]][0]) +', "to": ' + str(artists_by_id[pair[1]][0]) + '},\n')
                break
        
f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')
f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')
f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')

f.write("]")
f.close()

In [ ]:
most_collab_artists = [a for (a,b) in artists_by_id.items()]

In [ ]:
most_collab_artists

In [ ]:
list(itertools.combinations(collabs[6], 2))[0][0]

In [ ]:
artists_by_id['J. Cole']

In [ ]:
f = open("nodes.txt", "w")

f.write("var nodes = new vis.DataSet([\n")

for artist, i in top100.items():
    f.write("\t{id: "+str(i[0])+", label: '"+artist+"'},\n") 
        
f.write("]);")
f.close()

In [ ]:
'Lil Wayne' in ['Lil Wayne', 'asdsd'] and 'Li' in ['Lil Wayne', 'asdsd']

In [ ]:
 for pair in list(itertools.combinations(list(top100.keys()), 2)):
        print(pair)

In [ ]:
artists_by_id['Lil Wayne'][0]

In [ ]:
f = open("edges.txt", "w")


f.write("var edges = new vis.DataSet([\n")

for pair in list(itertools.combinations(list(top100.keys()), 2)):
    for collab in collabs:
        if pair[0] in collab and pair[1] in collab:
                f.write("\t{from: "+str(artists_by_id[pair[0]][0])+', to: '+str(artists_by_id[pair[1]][0])+"},\n")
                break
        
f.write("]);")
f.close()